In [1]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [2]:
df = pd.read_csv("rawdata.csv")


features :
- brawler 1-6
- map

In [3]:
firstTeams = []
secondTeams = []

for index, row in df.iterrows():
    team1 = []
    team2 = []

    team1.append(row["Brawler1Name"])
    team1.append(row["Brawler2Name"])
    team1.append(row["Brawler3Name"])

    team2.append(row["Brawler4Name"])
    team2.append(row["Brawler5Name"])
    team2.append(row["Brawler6Name"])

    # sort teams that brawlers are in same order
    team1 = sorted(team1)
    team2 = sorted(team2)

    # concat teams to one string
    team1string = "_".join(team1)
    team2string = "_".join(team2)

    firstTeams.append(team1string)
    secondTeams.append(team2string)

'''combinedFirstTeams = []
combinedSecondTeams = []

for i in range(len(firstTeams)):
    team1string = ""
    team1string = team1string + firstTeams[i][0]
    team1string = team1string + firstTeams[i][1]
    team1string = team1string + firstTeams[i][2]
    combinedFirstTeams.append(team1)
    team2string = ""
    team2string = team2string + secondTeams[i][0]
    team2string = team2string + secondTeams[i][1]
    team2string = team2string + secondTeams[i][2]
    combinedSecondTeams.append(team2)'''

'combinedFirstTeams = []\ncombinedSecondTeams = []\n\nfor i in range(len(firstTeams)):\n    team1string = ""\n    team1string = team1string + firstTeams[i][0]\n    team1string = team1string + firstTeams[i][1]\n    team1string = team1string + firstTeams[i][2]\n    combinedFirstTeams.append(team1)\n    team2string = ""\n    team2string = team2string + secondTeams[i][0]\n    team2string = team2string + secondTeams[i][1]\n    team2string = team2string + secondTeams[i][2]\n    combinedSecondTeams.append(team2)'

In [4]:
dfFinal = pd.DataFrame(columns=["Team1", "Team2", "map"])
dfFinal["Team1"] = firstTeams
dfFinal["Team2"] = secondTeams
dfFinal["map"] =df["map"].copy()
dfFinal["WinningTeam"] = df["WinningTeam"]

encoding = LabelEncoder()
encoding.fit(firstTeams + secondTeams)
dfFinal["Team1"] = encoding.transform(dfFinal["Team1"].values)
dfFinal["Team2"] = encoding.transform(dfFinal["Team2"].values)

# encode maps
encoding = LabelEncoder()
encoding.fit(dfFinal["map"].values)
dfFinal["map"] = encoding.transform(dfFinal["map"].values)

dfFinal.head(20)

,Team1,Team2,map,WinningTeam
0,172,73,0,1
1,172,949,0,1
2,172,456,0,1
3,172,865,0,1
4,172,459,0,1
5,172,497,0,1
6,225,172,0,2
7,963,172,0,2
8,169,172,0,2
9,976,175,11,1


In [5]:
# extract features
x_features = []
winners = []

for index, row in dfFinal.iterrows():
    features = []
    features.append(row["map"])
    features.append(row["Team1"])
    features.append(row["Team2"])

    x_features.append(features)
    winners.append(row["WinningTeam"])

In [10]:
tree2 = RandomForestClassifier()
tree2 = tree2.fit(x_features, winners)

In [13]:
# test tree by cross validation
scores = cross_val_score(tree2, x_features, winners, scoring='accuracy')
np.mean(scores)

0.6116612872603

In [14]:
scores = cross_val_score(tree2, x_features, winners, scoring='accuracy')
acc = np.mean(scores)

print("Accuracy: {0}".format(acc))

Accuracy: 0.6020808809569015
